Dependencies

In [ ]:
!pip install pydub
from google.colab import drive
drive.mount('/content/gdrive')
from pathlib import Path
import os
import cv2
from multiprocessing import Pool
from pydub import AudioSegment
from shutil import copy
from google.colab.drive import flush_and_unmount
!pip install audiomentations[extras]
from audiomentations import (Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, AddBackgroundNoise, AddShortNoises, ClippingDistortion, Gain, Mp3Compression)
import numpy as np
import librosa
import soundfile as sf
import random

Install ffmpeg

In [ ]:
!update-alternatives --config python3
!add-apt-repository ppa:savoury1/ffmpeg4
!apt install ffmpeg
!ffmpeg -version

Directories

In [ ]:
recordings_dir = Path("/content/gdrive/MyDrive/FYP/Top_Seven_Full_Even")
split_wav_dir = Path("/content/gdrive/MyDrive/FYP/Consistent_dataset/Split_validation")
even_clips_dir = Path(f"/content/gdrive/MyDrive/FYP/Consistent_dataset/Even_Split_validation")
# augmented_dir = Path("/content/gdrive/MyDrive/FYP/Consistent_dataset/Augmented_train")
image_dir = Path("/content/gdrive/MyDrive/FYP/Consistent_dataset/Even_Split_validation_images")

Convert mp3s to wav, if any

In [ ]:
for mp3_path in recordings_dir.glob('**/*.mp3'):
  src_mp3_absolute = str(mp3_path.absolute())
  print(f'Converting {src_mp3_absolute}')
  species = str(mp3_path).split('/')[-2]
  dst_wav_absolute = str(recordings_dir) + "/" + species + "/" + mp3_path.stem + ".wav"
  Path(dst_wav_absolute).parent.mkdir(parents=True, exist_ok=True)
  try:
    sound = AudioSegment.from_mp3(src_mp3_absolute)
    sound.export(dst_wav_absolute, format="wav")
  except Exception:
    pass
  os.remove(mp3_path)

flush_and_unmount()
drive.mount('/content/gdrive')

mp3_list = []
for mp3_path in recordings_dir.glob('**/*.mp3'):
  mp3_list.append(mp3_path)
assert mp3_list == []

Seconds to Min:Sec

In [ ]:
def seconds_to_min_sec(seconds):
  mins = int(seconds/60)
  seconds = seconds - (60*mins)
  if seconds < 10:
    seconds = f"0{seconds}"
  if mins < 10:
    mins = f"0{mins}"
  return f"{mins}:{seconds}"

Split up audio files

In [ ]:

  print(f"Working on {file.stem}")
  species = str(file.parents[0]).split('/')[-1]
  Path(f"{split_wav_dir}/{species}").mkdir(parents=True, exist_ok=True)
  os.system(f"ffmpeg -i \"{file.absolute()}\" -af loudnorm \"{file.stem}temp.wav\" -y")
  os.system(f'ffmpeg -i \"{file.stem}temp.wav\" -af \"highpass=f=20, lowpass=f=9500\" \"{file.stem}temp1.wav\" -y')
  os.system(f"ffmpeg -i \"{file.stem}temp1.wav\" -af afftdn \"{file.stem}temp2.wav\" -y")
  os.system(f"ffmpeg -i \"{file.stem}temp2.wav\" -af silenceremove=stop_periods=-1:stop_duration=1:stop_threshold=-46dB \"{file.stem}temp3.wav\" -y")
  # os.system(f"ffmpeg -i \"{file.stem}temp3.wav\" -f segment -segment_time 3 -c copy \"{split_wav_dir}/{species}/{file.stem}%03d.wav\" -y")
  total_duration = AudioSegment.from_file(file).duration_seconds
  overlap = 0.25
  start = 0
  duration = 3
  count = 0def split_wav(file):
  while start < total_duration:
    start_mins = seconds_to_min_sec(start)
    end = seconds_to_min_sec(start+duration)
    # print(f"start = {start_mins}")
    # print(f"end = {end}")
    command = f"ffmpeg -i \"{file.stem}temp3.wav\" -ss {start_mins} -to {end} -acodec copy -y \"{split_wav_dir}/{species}/{file.stem}_{count}.wav\" -y"
    os.system(command)
    count += 1
    start += duration*(1-overlap)

  if Path(f"{file.stem}temp.wav").exists() is False or Path(f"{file.stem}temp1.wav").exists() is False or Path(f"{file.stem}temp2.wav").exists() is False or Path(f"{file.stem}temp3.wav").exists() is False:
    raise FileNotFoundError
  os.remove(f"{file.stem}temp.wav")
  os.remove(f"{file.stem}temp1.wav")
  os.remove(f"{file.stem}temp2.wav")
  os.remove(f"{file.stem}temp3.wav")
  #
  for split_file in Path(f"{split_wav_dir}/{species}").iterdir():
    if split_file.name.startswith(file.stem):
        audio = AudioSegment.from_file(split_file)
        if audio.duration_seconds < 2.9:
          os.remove(split_file)
  #


files = []
for file in recordings_dir.glob('**/*.wav'):
  files.append(file)

pool = Pool(os.cpu_count())
pool.map(split_wav, files)

Print the number of clips per species

In [ ]:
species_to_count = {}
for file in split_wav_dir.glob('**/*.wav'):
#for file in recordings_dir.glob('**/*.wav'):
  species = str(file.parents[0]).split('/')[-1]
  species_to_count[species] = 1 if species not in species_to_count  else species_to_count[species] + 1 

species_to_count = {k: v for k, v in sorted(species_to_count.items(), key=lambda item: item[1], reverse=True)}

for species, count in species_to_count.items():
  print(f"{species} : {count}")

(Optional) Make evenly distributed / set number of clips per species

In [ ]:
species_list = species_to_count.keys()

number_to_copy = min([species_to_count[species] for species in species_list])

# number_to_copy = 747

print(number_to_copy)

species_to_files = {}
for species in species_list:
  species_to_files[species] = []

for file in split_wav_dir.glob('**/*.wav'):
# for file in recordings_dir.glob('**/*.wav'):
  species = str(file.parents[0]).split('/')[-1]
  if species in species_list and len(species_to_files[species]) < number_to_copy:
    species_to_files[species].append(file)

count = 1
for species, file_list in species_to_files.items():
  for file in file_list:
    copy_species_path = Path(f"{even_clips_dir}/{species}")
    copy_species_path.mkdir(exist_ok=True, parents=True)
    print(f"copying {file.stem} ({count}/{number_to_copy*len(species_to_files)})")
    output_path = Path(f"{copy_species_path}/{file.name}")
    copy(file, output_path)
    count += 1
  flush_and_unmount()
  drive.mount('/content/gdrive')

Apply Augmentation of audio files

In [ ]:

MIN_AUDIO_DURATION_SECONDS = 15
MAX_NUMBER_OF_SPLITS_TO_MAKE = 7
MIN_NUMBER_OF_SPLITS_TO_MAKE = 3

def make_audio_files(file_path_str, augmented_dir, species):
    duration = librosa.get_duration(filename=file_path_str)
    output_dir = f"{str(augmented_dir)}/{species}"
    Path(output_dir).mkdir(exist_ok=True, parents=True)
    output = f"{str(output_dir)}/{Path(file_path_str).name}"
    copy(file_path_str, output)
    files = [output]
    if duration <= MIN_AUDIO_DURATION_SECONDS:
      return files
    for split_number in range(random.randrange(MIN_NUMBER_OF_SPLITS_TO_MAKE, MAX_NUMBER_OF_SPLITS_TO_MAKE)):
        start = random.uniform(0, duration - MIN_AUDIO_DURATION_SECONDS)
        end = random.uniform(start + MIN_AUDIO_DURATION_SECONDS, duration)
        start_mins = seconds_to_min_sec(start)
        end_mins = seconds_to_min_sec(end)
        file_path = Path(file_path_str)
        output_file_str = f"{str(augmented_dir)}/{species}/{file_path.stem}_{split_number}{file_path.suffix}"
        command = f"ffmpeg -i \"{file_path_str}\" -ss {start_mins} -to {end_mins} -acodec copy -y \"{output_file_str}\" -y"
        print(command)
        os.system(command)
        if Path(output_file_str).exists() is False:
            raise FileNotFoundError
        files.append(output_file_str)
    return files

operations = [AddGaussianNoise(max_amplitude=0.03, p=0.75), AddGaussianNoise(min_amplitude=0.002), 
              Shift(min_fraction=-1, max_fraction=1, p=0.75), AddBackgroundNoise("/content/gdrive/MyDrive/FYP/Ambient-Sounds", p=0.75), 
              AddShortNoises("/content/gdrive/MyDrive/FYP/Ambient-Sounds"), ClippingDistortion(min_percentile_threshold=20, max_percentile_threshold=30),
              Gain(min_gain_in_db=-20, max_gain_in_db=20, p=0.625), Mp3Compression()]

# for file in even_clips_dir.glob('**/*.wav'):
for file in recordings_dir.glob('**/*.wav'):
  species = str(file.parents[0]).split('/')[-1]
  audio_files_list = make_audio_files(str(file), augmented_dir, species)
  print(f" --- Making splits for {file.stem}")
  for audio_file_path in audio_files_list:
    print(f" ------ Augmenting {Path(audio_file_path).stem}")
    signal, sample_rate = librosa.load(str(audio_file_path))
    for operation in operations:
      print(f" --------- Applying operation {type(operation).__name__}")
      augment = Compose([operation])
      augmented_samples = augment(samples=signal, sample_rate=sample_rate)
      Path(f"{augmented_dir}/{species}").mkdir(exist_ok=True, parents=True)
      sf.write(f"{augmented_dir}/{species}/{Path(audio_file_path).stem}_{type(operation).__name__}.wav", augmented_samples, sample_rate)

Make Images

In [ ]:
from multiprocessing import Pool

def make_image(file_path):
  species = str(file_path.parents[0]).split('/')[-1]
  Path(f"{image_dir}/{species}").mkdir(exist_ok=True, parents=True)
  image_file_path_str = f"{image_dir}/{species}/{file_path.stem}.png"
  if Path(image_file_path_str).exists() and cv2.imread(image_file_path_str).shape == (968, 602, 3):
    return
  print(f"Making image for {file_path.stem}")
  os.system(f"ffmpeg -i \"{file_path.absolute()}\" -lavfi showspectrumpic=s=600x960:stop=10000 \"{image_file_path_str}\"")
  img = cv2.imread(image_file_path_str)
  crop_img = img[60:-60, 156:-156]
  cv2.imwrite(image_file_path_str, crop_img)

print(os.cpu_count())

file_list = []
for file in even_clips_dir.glob('**/*.wav'):
# for file in recordings_dir.glob('**/*.wav'):
# for file in augmented_dir.glob('**/*.wav'):
  file_list.append(file)

pool = Pool(os.cpu_count())
pool.map(make_image, file_list)
print("Done")

4
Done
